Get the Isomeric Smiles from a list of CAS-ID's

In [1]:
import pandas as pd
from selenium import webdriver
import time
supplier = 'SantaCruztBioTech'

In [2]:
# df = pd.read_excel('Stock list Carbohydrates.xlsx', dtype=str)
df = pd.read_excel(supplier+' Stock list Carbohydrates.xlsx', dtype=str)
df = df.fillna('')    
print(df.shape)
print(df.columns)
# info = [(j,k,l) for i,j,k,l in zip(df['Storage Location US'],df['Material Code'],df['Name'],df['CAS']) if i == 'Yes' and str(l) != '']
info = [(j,k,l) for j,k,l in zip(df['CATALOG #'],df['PRODUCT NAME/APPLICATION'],df['CAS']) if str(l) != '']
print(info[:10])

(682, 3)
Index(['CATALOG #', 'PRODUCT NAME/APPLICATION', 'CAS'], dtype='object')
[('sc-221517', 'D-Ribonolactone', '5336-08-3'), ('sc-222394', 'Tyvelose', '5658-12-8'), ('sc-221100', '6-Deoxy-D-glucose', '7658-08-4'), ('sc-217338', '6-Deoxy-L-talose', '7658-10-8'), ('sc-223855', 'Calcium alpha-D-heptagluconate dihydrate', '10030-53-2'), ('sc-218620', 'L-(-)-Mannose', '10030-80-5'), ('sc-207810', 'L-Rhamnose monohydrate', '10030-85-0'), ('sc-206247', '1,3,4,6-Tetra-O-acetyl-2-amino-2-deoxy-alpha-D-glucopyranose Hydrochloride', '10034-19-2'), ('sc-213795', '2-Deoxyribose 5-phosphate sodium salt', '102916-66-5'), ('sc-220864', '3-Deoxy-D-manno-2-octulosonic Acid, Ammonium Salt', '103404-70-2')]


In [3]:
driver = webdriver.Firefox()

url = f'https://pubchem.ncbi.nlm.nih.gov/#query='
driver.get(url)
time.sleep(10)


isomericSMILES = []

for i,j,k in info:
    url = f'https://pubchem.ncbi.nlm.nih.gov/#query={k}'
    driver.get(url)
    print(k)
    for _ in range(10):
        time.sleep(1)
        html = driver.page_source
        x = html[html.find('Isomeric SMILES'):]
        x = x[:x.find('&nbsp;&nbsp')]
        x = x[x.find('breakword'):]
        y = x[x.find('<span>')+6:x.find('</span>')]
        if y != '':
            print(k,y)
            isomericSMILES += [[y,i,j,k]]
            break

df2 = pd.DataFrame(isomericSMILES,columns=['SMILES','Material Code','Name','CAS'])
df2.to_csv(supplier+'StockListCarbohydratesFiltered.csv')
print(df2)

driver.close()

5336-08-3
5336-08-3 C([C@@H]1[C@H]([C@H](C(=O)O1)O)O)O
5658-12-8
5658-12-8 C[C@H]([C@H](C[C@@H](C=O)O)O)O
7658-08-4
7658-08-4 C[C@H]([C@H]([C@@H]([C@H](C=O)O)O)O)O
7658-10-8
7658-10-8 C[C@@H]([C@H]([C@H]([C@H](C=O)O)O)O)O
10030-53-2
10030-53-2 C([C@H]([C@H]([C@@H]([C@H]([C@H](C(=O)[O-])O)O)O)O)O)O.C([C@H]([C@H]([C@@H]([C@H]([C@H](C(=O)[O-])O)O)O)O)O)O.O.O.[Ca+2]
10030-80-5
10030-80-5 C([C@@H]([C@@H]([C@H]([C@H](C=O)O)O)O)O)O
10030-85-0
10030-85-0 C[C@@H]([C@@H]([C@H]([C@H](C=O)O)O)O)O.O
10034-19-2
10034-19-2 CC(=O)OC[C@@H]1[C@H]([C@@H]([C@H]([C@H](O1)OC(=O)C)N)OC(=O)C)OC(=O)C.Cl
102916-66-5
102916-66-5 C(C=O)[C@@H]([C@@H](COP(=O)(O)[O-])O)O.[Na+]
103404-70-2
103404-70-2 C1C(C(C(OC1(C(=O)O)O)C(CO)O)O)O.N
10593-29-0
10593-29-0 C([C@@H]1[C@H]([C@@H]([C@H]([C@@H](O1)[S-])O)O)O)O.[Na+]
106032-60-4
106032-60-4 [2H]C1(C(C(C(OC1O)CO)O)O)O
106032-61-5
106032-61-5 [2H]C1([C@@H]([C@H]([C@@H]([C@H](O1)CO)O)O)O)O
106032-62-6
106032-62-6 [13CH2]([C@H]([C@H]([C@@H]([C@H](C=O)O)O)O)O)O
108342-85-4
108

In [4]:
# df = pd.read_excel('Stock list Carbohydrates.xlsx')
# df = df[df['Storage Location US'] == 'Yes']
# df.reset_index(inplace = True)
# casses = df['CAS']
# names = df['Name']
# codes = df['Material Code']

# test_list = []
# for x,y,z in zip(casses,names,codes):
#     test_list.append([cirpy.resolve(str(x), 'smiles', ['cas_number']),y,x,z])
# test_list = pd.DataFrame([i for i in test_list if i[0]], columns =['SMILES','Name','cas accession','material code'])
# test_list.to_csv('StockListCarbohydratesCirpy.csv')
# print(test_list)
# print(test_list.shape)